<font size='10'><h1 align="center"><font color='blue'>iPy oxDNA Examples</font></h1>

<center>
<img src="oxDNA.png">
</center>

<font size='6'><a class="anchor" id="3"></a> <h2 align="center"> <font color='black'>Umbrella Sampling</font></h2> 

<font size='3'><font color='black'>Utilizing the ComUmbrellaSampling class, we can automatically run 1D umbrella sampling simulations with a COM order parameter.</font>

In [ ]:
from umbrella_sampling import ComUmbrellaSampling, MeltingUmbrellaSampling
from oxdna_simulation import SimulationManager, Simulation, Observable
import os
import matplotlib.pyplot as plt

In [ ]:
simulation_manager = SimulationManager() 
simulation_manager.start_nvidia_cuda_mps_control(pipe='temp')

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples')
file_dir = f'{path}/duplex_melting'
sim_dir = f'{file_dir}/unbiased'

sim = Simulation(file_dir, sim_dir)

In [ ]:
sim.build(clean_build='force')
sim.input_file({'steps':'1e7','print_energy_every': '1e6', 'print_conf_interval':'1e6', "CUDA_list": "no", 'use_edge': 'false'})
sim.oxpy_run.run(subprocess=True)

In [ ]:
sim.analysis.plot_energy()
sim.analysis.view_init()

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples')
file_dir = f'{path}/duplex_melting'
system = 'us_melting'

com_list = '8,9,10,11,12,13,14,15'
ref_list = '0,1,2,3,4,5,6,7'
xmin = 0
xmax = 8
n_windows = 95

stiff = 0.8

equlibration_parameters = {'steps':'1e6','print_energy_every': '5e6',
                           'print_conf_interval':'5e6', "CUDA_list": "no",
                           'use_edge': 'false', 'refresh_vel': '1',
                           'fix_diffusion': '0', 'fix_diffusion_every': '1000'
                          }

production_parameters ={'steps':'1e7','print_energy_every': '2e7',
                        'print_conf_interval':'2e7', "CUDA_list": "no",
                        'use_edge': 'false', 'refresh_vel': '1',
                        'fix_diffusion': '0', 'fix_diffusion_every': '1000'
                        }

us = MeltingUmbrellaSampling(file_dir, system)
simulation_manager = SimulationManager()

In [ ]:
us.build_equlibration_runs(simulation_manager, n_windows, com_list, ref_list, stiff, xmin, xmax, equlibration_parameters,
                           observable=True, sequence_dependant=True, print_every=1e3, name='com_distance.txt')

In [ ]:
simulation_manager.run()

In [ ]:
us.build_production_runs(simulation_manager, n_windows, com_list, ref_list, stiff, xmin, xmax, production_parameters,
                           observable=True, sequence_dependant=True, print_every=1e3, name='com_distance.txt')

In [ ]:
simulation_manager.run()

In [ ]:
us.analysis.view_conf('eq', 46)

In [ ]:
us.com_distance_observable(com_list, ref_list)
us.hb_list_observable(print_every=1e3)
plt.figure(dpi=200)
for idx in range(93,95,1):
    us.analysis.hist_observable('prod', idx, observable=us.observables_list[0], bins=200)#, sliding_window=5)
# plt.legend(fontsize=4)

In [ ]:
for idx in range(0,48):
    print(idx)
    us.equlibration_sims[idx].sim_files.last_conf

In [ ]:
us.analysis.view_conf('eq', 13)

In [ ]:
wham_dir = os.path.abspath('/scratch/mlsample/ipy_oxDNA/wham/wham')
n_bins = '200'
tol = '1e-7'
n_boot = '30'
us.wham_run(wham_dir, xmin, xmax, stiff, n_bins, tol, n_boot)

In [ ]:
us.plot_free()

In [ ]:
path = os.path.abspath('/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples')
file_dir = f'{path}/duplex_melting'
systems = ['us_melting_10', 'us_melting', 'us_melting_30','us_melting_40']

us_list = [MeltingUmbrellaSampling(file_dir, system) for system in systems]

In [ ]:
us_list[0].fig_ax()

for us in us_list:
    us.plot_free(ax=us_list[0].ax)
plt.legend([f'{10 + 273}K', f'{20 + 273}K', f'{30 + 273}K', f'{40 + 273}K'])
plt.title('8nt Duplex Melting Free Energy Profile')
plt.xlabel('COM Distance (nm)')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
free = pd.read_csv(f'{us.system_dir}/production/com_dir/freefile', sep='\t')

In [ ]:
free_vals = free.iloc[-95:].Free

In [ ]:
free_vals.iloc[1:]

In [ ]:
H_vals = np.zeros(94)
us.hb_list_observable(print_every=1e3)
for idx, sim in enumerate(us.production_sims):
    sim_dir = sim.sim_dir
    file_name = us.observables_list[1]['output']['name']
    df = pd.read_csv(f"{sim_dir}/{file_name}", header=None)
    df = np.concatenate(np.array(df))
    H, bins = np.histogram(df, density=False, bins=np.linspace(0, 8, 95))
    H_vals += H

In [ ]:
prob_H = H_vals / sum(H_vals)

In [ ]:
plt.plot(bins[:-1], -np.log(prob_H))

In [ ]:
file_name = us.observables_list[0]['output']['name']